# Connecting to and starting TigerGraph service
Make sure to increase RAM available first

```ssh -p 14022 tigergraph@localhost```

Password: tigergraph

Once sshed into the container, run

```gadmin start all```

In [ ]:
import pandas as pd
import pyTigerGraph as tg

In [ ]:
conn = tg.TigerGraphConnection(host="http://localhost")

In [ ]:
print(conn.gsql("DROP ALL", options=[]))

In [ ]:
conditions = pd.read_csv("data/conditions.csv", names=["dateStart", "dateEnd", "patient", "condition", "code", "description"])
medications = pd.read_csv("data/medications.csv", names=["dateStart", "dateEnd", "patient", "medication", "code" , "description", "cost", "dispenses", "totalCost", "reasonCode", "reasonDesc"])
patients = pd.read_csv("data/patients.csv", names=["id","birthDate","deathDate","ssn","drivers","passport","prefix","first","last","suffix","maiden","marital","race","ethnicity","gender","birthPlace","address","city","state","zip"])
procedures = pd.read_csv("data/procedures.csv", names=["date", "patient", "encounter", "code", "description", "cost", "reasonCode", "reasonDescription"])
providers = pd.read_csv("data/providers.csv", names=["id","organization","name","gender","specialty","address","city","state","zip","utilization"])
encounters = pd.read_csv("data/encounters.csv", names=["id","dateStart","dateStop","patient","provider","encounterClass","code","description","cost","reasonCode","reasonDescription"])

In [ ]:
patients.head()

In [ ]:
providers.head()

In [ ]:
conditions.head()

In [ ]:
medications.head()

In [ ]:
procedures.head()

In [ ]:
encounters.head()

Figure out how you would go about finding what patients have seen a given provider. Look in the encounters table, obviously. But what about if you wanted to know the patient's birthdate and medications, along with what provider they are seeing? Thats like 2 joins between 3 tables. On an enterprise scale of patients, this is *really* slow. Lets model these tables into a graph database.

# Creating Vertices
We are going to create vertices for our data to reside on. We will then join these vertices together with edges.

In [ ]:
print(conn.gsql('''
    CREATE VERTEX Patient (PRIMARY_ID Id STRING, FirstName STRING, LastName STRING, Prefix STRING, Suffix STRING, MaidenName STRING, MaritalStatus STRING, Gender STRING, Race STRING, Ethnicity STRING, DateOfBirth DATETIME, DateOfDeath DATETIME DEFAULT "2999-12-31 00:00:00", SSN STRING, DL STRING, Passport STRING )
''', options=[]))

In [ ]:
print(conn.gsql('''
    CREATE VERTEX Provider (PRIMARY_ID Id STRING, Name STRING, Gender STRING, Speciality STRING, Utilization INT)
''', options=[]))

In [ ]:
print(conn.gsql('''
    CREATE VERTEX Condition (PRIMARY_ID Id STRING, Code STRING, Description STRING)
''', options=[]))

In [ ]:
print(conn.gsql('''
    CREATE VERTEX Medication(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, Dispense INT, TotalCost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
''', options=[]))

In [ ]:
print(conn.gsql('''
    CREATE VERTEX PatientProcedure(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
''', options=[]))

In [ ]:
print(conn.gsql('''
    CREATE VERTEX Visit (PRIMARY_ID Id STRING, Code STRING, Description STRING, Started DATETIME, Stopped DATETIME, Cost DOUBLE) 
''', options=[]))

# Adding Edges
Now here comes what makes a graph database special - predetermined relationships between different entities. We are going to add *edges* between are vertices, so that we don't have to perform joins when manipulating the data

In [ ]:
# Create Edge Between Patients and Visits, Providers and Visits, and Patients and Providers

print(conn.gsql('''
    CREATE DIRECTED EDGE patientVisit (FROM Patient, TO Visit) WITH REVERSE_EDGE = "patientSeen"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE providerVisit (FROM Provider, TO Visit) WITH REVERSE_EDGE = "providerSeen"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE patientSeesProvider (FROM Patient, TO Provider) WITH REVERSE_EDGE = "providerSeesPatient"
''', options=[]))

In [ ]:
# Create edge between Patients and Medications

print(conn.gsql('''
    CREATE DIRECTED EDGE hasMedication (FROM Patient, TO Medication, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientMedication"
''', options=[]))

In [ ]:
# Create edge between Patients and Conditions

print(conn.gsql('''
    CREATE DIRECTED EDGE hasCondition (FROM Patient, TO Condition, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientCondition"
''', options=[]))

In [ ]:
# Create edge between Patients and Procedures as well as Visits and Procedure

print(conn.gsql('''
    CREATE DIRECTED EDGE hasProcedure (FROM Patient, TO PatientProcedure, Performed DATETIME) WITH REVERSE_EDGE = "patientProcedure"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE procedureDuringVisit (FROM PatientProcedure, TO Visit, Performed DATETIME) WITH REVERSE_EDGE = "visitForProcedure"
''', options=[]))

# Tying it all together
After this, visit [here](http://localhost:14240/#/schema-designer) to see the completed schema

In [ ]:
print(conn.gsql("ls", options=[]))

In [ ]:
print(conn.gsql('''
    CREATE GRAPH Synthea(Patient, Provider, Condition, Medication, PatientProcedure, Visit, patientVisit, patientSeen, providerVisit, providerSeen, patientSeesProvider, providerSeesPatient, hasMedication, patientMedication, hasCondition, patientCondition, hasProcedure, patientProcedure, procedureDuringVisit, visitForProcedure)
''', options=[]))

# Loading Data
We could define loading jobs, but the easiest way is to use pyTigerGraph's loading functions.

In [ ]:
# First, need to set graph name in connection

conn.graphname = "Synthea"

In [ ]:
patients.head()

In [ ]:
'''
CREATE VERTEX Patient (PRIMARY_ID Id STRING, FirstName STRING, LastName STRING, Prefix STRING, Suffix STRING, MaidenName STRING, MaritalStatus STRING, Gender STRING, Race STRING, Ethnicity STRING, DateOfBirth DATETIME, DateOfDeath DATETIME DEFAULT "2999-12-31 00:00:00", SSN STRING, DL STRING, Passport STRING )
'''

def upsertPatients(row):
    attributes = {
        "FirstName": row["first"],
        "LastName": row["last"], 
        "Prefix": row["prefix"],
        "Suffix": row["suffix"], 
        "MaidenName": row["maiden"],
        "MaritalStatus": row["marital"],
        "Gender": row["gender"],
        "Race": row["race"],
        "Ethnicity": row["ethnicity"],
        "DateOfBirth": row["birthDate"],
        "SSN": row["ssn"],
        "DL": row["drivers"],
        "Passport": row["passport"]
    }
    conn.upsertVertex("Patient", row["id"], attributes)

In [ ]:
patients["deathDate"].fillna("2999-12-31 00:00:00", inplace=True)
patients.fillna("", inplace=True)

In [ ]:
patients.apply(lambda row: upsertPatients(row), axis=1)

In [ ]:
providers.head()

In [ ]:
# CREATE VERTEX Provider (PRIMARY_ID Id STRING, Name STRING, Gender STRING, Speciality STRING, Utilization INT)
def upsertProviders(row):
    attributes = {
        "Name": row["name"], 
        "Gender": row["gender"],
        "Speciality": row["specialty"],
        "Utilization": row["utilization"]
    }
    conn.upsertVertex("Provider", row["id"], attributes)

In [ ]:
providers.apply(lambda row: upsertProviders(row), axis=1)

In [ ]:
encounters.head()

In [ ]:
# CREATE VERTEX Visit (PRIMARY_ID Id STRING, Code STRING, Description STRING, Started DATETIME, Stopped DATETIME, Cost DOUBLE) 
def upsertVisits(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Started": row["dateStart"],
        "Stopped": row["dateStop"],
        "Cost": row["cost"]
    }

    conn.upsertVertex("Visit", row["id"], attributes)
    conn.upsertEdge("Patient", row["patient"], "patientVisit", "Visit", row["id"])
    conn.upsertEdge("Provider", row["provider"], "providerVisit", "Visit", row["id"])
    conn.upsertEdge("Patient", row["patient"], "patientSeesProvider", "Provider", row["provider"])

In [ ]:
encounters.apply(lambda row: upsertVisits(row), axis=1)

In [ ]:
medications.head()

In [ ]:
medications["dateEnd"].fillna("2999-12-31 00:00:00", inplace=True)
medications.fillna("", inplace=True)

In [ ]:
#CREATE VERTEX Medication(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, Dispense INT, TotalCost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
#CREATE DIRECTED EDGE hasMedication (FROM Patient, TO Medication, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientMedication"
def upsertMedications(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Cost": row["cost"],
        "Dispense": row["dispenses"],
        "TotalCost": row["totalCost"],
        "ReasonCode": str(row["reasonCode"]),
        "ReasonDescription": row["reasonDesc"]
    }

    conn.upsertVertex("Medication", row["code"], attributes)

    edgeAttributes = {
        "Started": row["dateStart"],
        "Stopped": row["dateEnd"]
    }

    conn.upsertEdge("Patient", row["patient"], "hasMedication", "Medication", row["code"])

In [ ]:
medications.apply(lambda row: upsertMedications(row), axis=1)

In [ ]:
conditions.head()

In [ ]:
len(conditions["condition"].unique())

In [ ]:
conditions.shape

In [ ]:
conditions["dateEnd"].fillna("2999-12-31 00:00:00", inplace=True)
conditions.fillna("", inplace=True)

In [ ]:
# CREATE VERTEX Condition (PRIMARY_ID Id STRING, Code STRING, Description STRING)
# CREATE DIRECTED EDGE hasCondition (FROM Patient, TO Condition, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientCondition"
def upsertConditions(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"]
    }

    conn.upsertVertex("Condition", row["code"], attributes)
    conn.upsertEdge("Patient", row["patient"], "hasCondition", "Condition", row["code"])

In [ ]:
conditions.apply(lambda row: upsertConditions(row), axis=1)

In [ ]:
procedures.head()

In [ ]:
procedures.fillna("", inplace=True)

In [ ]:
# CREATE VERTEX PatientProcedure(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
# CREATE DIRECTED EDGE hasProcedure (FROM Patient, TO PatientProcedure, Performed DATETIME) WITH REVERSE_EDGE = "patientProcedure"
# CREATE DIRECTED EDGE procedureDuringVisit (FROM PatientProcedure, TO Visit, Performed DATETIME) WITH REVERSE_EDGE = "visitForProcedure"

def upsertProcedure(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Cost": row["cost"],
        "ReasonCode": str(row["reasonCode"]),
        "ReasonDescription": row["reasonDescription"]
    }

    conn.upsertVertex("PatientProcedure", row["code"], attributes)

    edgeAttributes = {
        "Performed": row["date"]
    }

    conn.upsertEdge("Patient", row["patient"], "hasProcedure", "PatientProcedure", row["code"], edgeAttributes)
    conn.upsertEdge("PatientProcedure", row["code"], "procedureDuringVisit", "Visit", row["encounter"], edgeAttributes)

In [ ]:
procedures.apply(lambda row: upsertProcedure(row), axis=1)